# **VisL3D** example for many spectral lines

This notebook shows different ways to create 3D visualisations with many spectral lines with **VisL3D**.

We will use *vis.prep_mult* and *vis.prep_overlay* to obtain the Cube objects and we will also create the Cube object directly.

In [1]:
from visl3d import visual as vis
from visl3d import misc
import astropy.units as u
from spectral_cube import SpectralCube
import numpy as np

## Multiple lines from FITS

Create a visualisation with multiple lines by giving the spectral (mandatory) and spatial (optional) limits for different subcubes.<br>

In [ ]:
lines = [r'$[NII]_1$', r'$H\alpha$', r'$[NII]_2$']
l_isolevels = [[20,40,60,80], [20,40,60,80], [20,40,60,80]]
spectral_lims = [[1500,1700],[1750,1800],[1850,2100]]
spatial_lims = [[[4,53],[5,50]], [[30,56],[3,43]], [[23,44],[2,50]]]

cube = vis.prep_mult('NII_Ha_fluxHRR_corr.fits', spectral_lims=spectral_lims, unit='percent', l_isolevels=l_isolevels, lines=lines, spatial_lims=spatial_lims, image2d='DSS2 Blue', im2dcolor='cividis')

In [ ]:
vis.createVis(cube, 'mult_fits', description=misc.cube_info(cube))

## Multiple lines from array

Create a visualisation with multiple spectral lines by giving the data array and the header instead of the FITS file. It is useful to make transformations to the data before creating the visualisation, e.g., to apply a mask.

Note that in this example the lines are not real.

In [8]:
file = SpectralCube.read('NII_Ha_fluxHRR_corr.fits')
header = file.header
data = file.unmasked_data[:].to_value()

In [9]:
lines = [r'$[NII]_1$', r'$H\alpha$', r'$[NII]_2$']
l_isolevels = [[20,40,60,80], [20,40,60,80], [20,40,60,80]]

cube = vis.prep_mult(data, spectral_lims=[[1500,1700],[1750,1800],[1850,2100]], header=header, unit='percent', l_isolevels=l_isolevels, lines=lines, spatial_lims=[[[4,53],[5,50]], [[30,56],[3,43]], [[23,44],[2,50]]], image2d='DSS2 Blue', im2dcolor='cividis')

In [10]:
vis.createVis(cube, 'mult_array', description=misc.cube_info(cube))

## Overlay from FITS, given lines

Create an overlay of multiple spectral lines from a FITS files by introducing the position of the spectral lines and the witdth to be plotted around the line.

In [11]:
lines = {'NII': (6583.45*u.Angstrom, 80), 'Ha': (6562.05*u.Angstrom, 80)}
l_isolevels = [[20,40,60,80], [20,40,60,80]]

cube = vis.prep_overlay('NII_Ha_fluxHRR_corr.fits', lines=lines, unit='percent', l_isolevels=l_isolevels, spatial_lims=[[[4,53],[5,50]], [[30,56],[3,43]]], image2d='DSS2 Blue', im2dcolor='cividis')

In [12]:
vis.createVis(cube, 'overlay_fits_lines', description=misc.cube_info(cube))

## Overlay from FITS, given spectral_lims

Create an overlay of multiple spectral lines from a FITS files by introducing the spectral (mandatory) and spatial (optional) limits for different subcubes.

In [13]:
speclims = [[1500,1700],[1750,1800],[1850,2100]]
l_isolevels = [[20,40,60,80], [20,40,60,80], [20,40,60,80]]

cube = vis.prep_overlay('NII_Ha_fluxHRR_corr.fits', spectral_lims=speclims, unit='percent', l_isolevels=l_isolevels, spatial_lims=[[[4,53],[5,50]], [[30,56],[3,43]], [[23,44],[2,50]]], image2d='DSS2 Blue', im2dcolor='cividis')

In [14]:
vis.createVis(cube, 'ovelay_fits_spectral', description=misc.cube_info(cube))

## Overlay with array

Create an overlay of multiple spectral lines from different FITS files by introducing the data as arrays in the Cube object.<br>
In this example we will overlay three lines, OIII, NII and $H\alpha$ available in two different FITS files.

First, we read the data from the FITS files (we use Spectral Cube).

In [2]:
cube1 = SpectralCube.read('OIII_fluxMRB_corr.fits')
cube2 = SpectralCube.read('NII_Ha_fluxHRR_corr.fits')

There are different ways to find the location of the lines and select a subcube.<br>
**ViSL3D** includes a function to find nearest pixel to a given line and the values of the spectral axis.<br>
We define the lines and use that function to find the slices of those lines.

In [3]:
line1 = 5006.843 #OIII 1
line2 = 6583.45 #NII 2
line3 = 6562.05 #Halpha

line1 = misc.find_nearest(cube1.spectral_axis, line1)
line2 = misc.find_nearest(cube2.spectral_axis, line2)
line3 = misc.find_nearest(cube2.spectral_axis, line3)
print(line1, line2, line3)

(np.float64(5006.789999999999), np.int64(3988)) (np.float64(6583.436399999999), np.int64(1985)) (np.float64(6562.008399999999), np.int64(1765))


Knowing the position of the line, we select 80 at each side of the line to create the subcube.
Then, we create a list with the subcubes.


Remember that the order of the cubes introduced in **ViSL3D** must be [RA, DAC, Z], that's why we transpose the subcubes.

In [4]:
hw = 80

oiii = cube1.unmasked_data[line1[1]-hw:line1[1]+hw,:,:].to_value().T
nii = cube2.unmasked_data[line2[1]-hw:line2[1]+hw,:,:].to_value().T
ha = cube2.unmasked_data[line3[1]-hw:line3[1]+hw,:,:].to_value().T

l_cubes = [oiii/np.nanmax(oiii)*100, nii/np.nanmax(nii)*100, ha/np.nanmax(ha)*100]

The next parameter we need for the Cube object is the coordinates.
We use all the field of view.

In [5]:
spatial = cube1.world_extrema.to_value()

The size of our overlay is twice 80 pixels. Therefore we need to calculate the coordinates of those pixels.<br>
Only the difference is important, the real values are not since all lines are overlaid.

In [6]:
spectral = cube1.header['CDELT3'] * hw * 2
spectral = np.array([0, spectral])

We join the spatial and spectral coordinates

In [7]:
coords = np.vstack([spatial, spectral])
coords = np.sort(coords)

In [8]:
coords

array([[ 0.02154709,  0.02494047],
       [ 0.42205955,  0.42509689],
       [ 0.        , 17.6       ]])

Now we create the rest of parameters:

In [21]:

l_isolevels = [[20,40,60,80], [20,40,60,80], [20,40,60,80]]
cmaps = ['Greens', 'Blues', 'Reds']
units = ['percent', cube1.header['CUNIT1'], cube1.header['CUNIT2'], cube1.header['CUNIT3']]
mags = ['Intensity', cube1.header['CTYPE1'], cube1.header['CTYPE2'], cube1.header['CTYPE3']]
delta = [cube1.header['CDELT1'], cube1.header['CDELT2'], cube1.header['CDELT3']]

# In this case the cube.lines attribute is only used to display them in the web interface; it is optional
lines = {'OIII': (line1*u.Angstrom, hw), 'NII': (line2*u.Angstrom, hw), r'H$\alpha$': (line3*u.Angstrom, hw)}

Finally, we create the cube object and the visualisation

In [22]:
cube = vis.Cube(l_cubes, coords, l_isolevels, cmaps, units, mags, delta, lines=lines)

vis.createVis(cube, 'overlay_array', description=misc.cube_info(cube))